In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pickle

# Load the dataset
dataset = pd.read_csv("Laptop_price.csv")

# Display the first 5 rows of the dataset
print(dataset.head(5))

# Clean and ensure the correct data types for categorical columns
dataset['Condition'] = dataset['Condition'].str.strip().replace('', np.nan).astype(str)
dataset['Brand'] = dataset['Brand'].str.strip().replace('', np.nan).astype(str)


# Drop rows with NaN values
dataset.dropna(inplace=True)


# Split the dataset into features and target variable
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Define the ColumnTransformer with OneHotEncoder for the categorical column
ct = ColumnTransformer(
    transformers=[
        ("encoder", OneHotEncoder(), [0, 6])  # Assuming column 0 & 6 is the categorical column
    ],
    remainder="passthrough",
)

# Apply the ColumnTransformer to the feature set
x = np.array(ct.fit_transform(x))

# Split the transformed dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# Train the Linear Regression model
regressor = LinearRegression()
regressor.fit(x_train, y_train)


# Function to encode a new sample and make predictions
def predict_new_sample(sample, ct, regressor):
    # Transform the new sample using the ColumnTransformer
    sample_encoded = np.array(ct.transform([sample]))
    # Make a prediction using the trained regressor
    prediction = regressor.predict(sample_encoded)
    return prediction


# New sample to predict (ensure this matches the format used in the dataset)
new_sample = [
    "Asus",   # Brand
    3.83,     # Weight
    16,       # RAM
    512,      # Storage
    11.18,    # Screen Size
    2.64,     # Processor Speed
    "New"     # Condition (categorical)
   
]

# Make a prediction for the new sample
prediction = predict_new_sample(new_sample, ct, regressor)
print("Prediction for the new sample:", prediction)

# Predict the target values for the test set
y_pred = regressor.predict(x_test)

# Calculate and print the R^2 score
#print("R^2 score:", r2_score(y_test, y_pred))

from sklearn.metrics import mean_squared_error

# Predict the target values for the test set with KNN
knn_y_pred = regressor.predict(x_test)

# Calculate R^2 and MSE for Linear Regression
linear_r2 = r2_score(y_test, y_pred)
linear_mse = mean_squared_error(y_test, y_pred)

# Calculate R^2 and MSE for KNN
knn_r2 = r2_score(y_test, knn_y_pred)
knn_mse = mean_squared_error(y_test, knn_y_pred)

# Print the results
print("Linear Regression R² score:", linear_r2)
print("Linear Regression MSE:", linear_mse)
print("KNN R² score:", knn_r2)
print("KNN MSE:", knn_mse)

# Save the trained regressor to a .pkl file
with open("model.pkl", "wb") as f:
    pickle.dump(regressor, f)

# Save the ColumnTransformer to a .pkl file
with open("ct.pkl", "wb") as f:
    pickle.dump(ct, f)

    Brand  Processor_Speed  RAM_Size  Storage_Capacity  Screen_Size    Weight  \
0    Asus         3.830296        16               512    11.185147  2.641094   
1    Acer         2.912833         4              1000    11.311372  3.260012   
2  Lenovo         3.241627         4               256    11.853023  2.029061   
3    Acer         3.806248        16               512    12.280360  4.573865   
4    Acer         3.268097        32              1000    14.990877  4.193472   

  Condition  Price  
0      Used  68000  
1       New  35000  
2      Used  32000  
3      Used  68000  
4       New  95000  
Prediction for the new sample: [58758.76864585]
Linear Regression R² score: 0.9046393450147558
Linear Regression MSE: 36663728.22504238
KNN R² score: 0.9046393450147558
KNN MSE: 36663728.22504238
